# How to Infere & Extract Data from a Pre-Trained YOLO Detection Model - Glove Tracking
---
If you have any questions, please contact the authors of the repository.

## Pre-work

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

## Clone BaseballCV Repo, set as Current Directory and Install Requirements

In [ ]:
!git clone https://github.com/dylandru/BaseballCV.git
%cd BaseballCV
!pip install -r requirements.txt
from ultralytics import YOLO

## Download Example Image from Dataset

**NOTE:** If you want to run inference using your own file as input, simply upload image to Google Colab and update `SOURCE_IMAGE_PATH` with the path leading to your file.

In [ ]:
from scripts.load_tools import load_dataset

load_dataset("/content/BaseballCV/datasets/yolo/baseball_rubber_home_glove.txt") #can be any dataset within files

In [ ]:
SOURCE_IMAGE_PATH = "baseball_rubber_home_glove/baseball_rubber_home_glove/train/images/0000196.jpg"
print("Source Image Updated.")

## Box Detection with Pre-Trained model (**Glove Detection Model**)

**NOTE:** This can be used with any type of detection model, although the process may slightly differ for other types of YOLO models.

In [ ]:
from scripts.load_tools import load_model

model_weights = load_model(model_alias='glove_tracking')
model = YOLO(model_weights)


In [ ]:
results = model.predict(source=SOURCE_IMAGE_PATH, save=True)

**NOTE:** By default, the results of each subsequent inference sessions are saved in `/runs/detect/predict`, in directories named `exp`, `exp2`, `exp3`, ... You can override this behavior by using the `--name` parameter.

In [ ]:
from IPython.display import Image

Image(filename="runs/detect/predict/0000196.jpg", width=600)

## Process Box Detections

**EXAMPLE:** Returning Box Predictions as Tensors

In [ ]:
for r in results: #access predictions from frame
  for box in r.boxes: #access individual boxes from frame
    print(box.xywh) #print x, y coordinate and width/height of box as tensor

**EXAMPLE:** Since Tensors are not always the most ideal data format, you can convert the box predictions to a numpy array.

In [ ]:
import numpy

for r in results: #access predictions from frame
  for box in r.boxes.cpu().numpy(): #access individual boxes from frame, move to cpu memory, convert to numpy array from tensor
    print(box.xywh) #print x, y coordinate and width/height of box as tensor

**EXAMPLE:** The output of the box prediction can be outputted in multiple formats, not just X, Y, W, H. Other aspects of the boxes can also be outputted.

In [ ]:
for r in results:
  for box in r.boxes.cpu().numpy():
    print(f"XYXY: {box.xyxy}")
    print(f"XYWHN (Normalized XYWH): {box.xywh}")
    print(f"XYXYN (Normalized XYXY): {box.xyxyn}")
    print(f"Confidence: {box.conf}")
    print(f"Track ID: {box.id}")
    print(f"Class Value: {box.cls} \n")

## Utilizing OpenCV to Visualize Predictions

**EXAMPLE:** Visualize Rectangles Individually on Each Image

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

for box in r.boxes.cpu().numpy():
    image = cv2.imread(SOURCE_IMAGE_PATH)
    x1, y1, x2, y2 = box.xyxy[0]
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    cv2_imshow(image)

**EXAMPLE:** Each object was able to be detected across images with the numerical output. Utilizing the other classes and some reconfiguration, this can be done on a singular image with the class name and confidence of class.

In [ ]:
image = cv2.imread(SOURCE_IMAGE_PATH)

for box in r.boxes.cpu().numpy():
    x1, y1, x2, y2 = box.xyxy[0]
    cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2) #draw rectangle

    class_id = int(box.cls.item())
    class_name = model.names[class_id] #get class name based on ID
    conf = float(box.conf.item())

    label = f"Class: {class_name}, Conf: {conf:.3f}"
    cv2.putText(image, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2) #put class and confidence

cv2_imshow(image)

## Inference on Video

In [ ]:
SOURCE_VIDEO_PATH = "/content/BaseballCV/assets/hunter_harvey_splitter.avi"
print("Source Video Updated.")

In [ ]:
results = model.predict(source=SOURCE_VIDEO_PATH, save=True)

In [ ]:
%cd /content/BaseballCV/runs/detect/predict
!ls

In [ ]:
import moviepy.editor

# Load the video file
video = moviepy.editor.VideoFileClip(filename="hunter_harvey_splitter.avi")
# Resize the video to a new resolution, e.g., (width, height)
resized_video = video.resize((640, 360))  # Example: resizing to 640x360
moviepy.editor.ipython_display(resized_video)

##**CONGRATS!** You utilized the output of the Glove Detector model to recreate the prediction on the image!